<a href="https://colab.research.google.com/github/manavmittal05/Large-Language-Models/blob/main/A1/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers huggingface_hub
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install bitsandbytes

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-9jq_mwmw
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-9jq_mwmw
  Resolved https://github.com/huggingface/transformers.git to commit 746104ba6f0514159d58dc2fb09c887d0e9d4863
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.45.0.dev0-py3-none-any.whl size=9614041 sha256=f6b72911459d993e7dca48909d3cd0322b096d0fd59232711f49ae0854c6c242
  Stored in directory: /tmp/pip-ephem-wheel-cache-eea98rtp/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.42.4
    Uninstalling transformers-4.42.4:
      Successfully uninstalled transformers-4.42.4
  Cloning https://github.com

### Llama3.1 Prompting and Setup

In [ ]:
from huggingface_hub import login
from google.colab import userdata

# Login with your API token
login(userdata.get('HF_TOKEN'))

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### Fact checking using llama3.1

In [ ]:
messages = [
    {"role": "user", "content": "Select two random numbers between 1381 and 1453 and multiply them together, reporting the result. "},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


I've randomly selected two numbers between 1381 and 1453:

Number 1: 1412
Number 2: 1419

Multiplying them together gives:

1412 × 1419 = 20,049,288


In [ ]:
messages = [
    {"role": "user", "content": "what the current price of gold in INR"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


The current price of gold in Indian Rupees (INR) can vary depending on the market conditions, global demand, and other factors. As of [current date], the price of gold in INR is:

* 10 grams of 22-karat gold: ₹45,500 - ₹46,500
* 10 grams of 24-karat gold: ₹48,500 - ₹49,500
* 1 gram of 24-karat gold: ₹4,850 - ₹4,950

Please note that these prices are approximate and can fluctuate depending on the market conditions. It's always a good idea to check with a reliable gold dealer or a financial website for the latest prices.

Some popular gold prices in INR can be found on websites such as:

* Gold Price India (goldpriceindia.com)
* A24M (a24m.com)
* Gold Rate Today (goldratetoday.com)
* Moneycontrol (moneycontrol.com)

Please keep in mind that these prices are subject to change and may not reflect the actual price at the time of your inquiry.


In [ ]:
messages = [
    {"role": "user", "content": "which is bigger, 9.11 and 9.9"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1]["content"])

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


A simple question!

9.11 is bigger than 9.9.


### OpenHathi Prompting and Setup

In [ ]:
import torch
import warnings
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

warnings.filterwarnings('ignore')

In [ ]:
# loading the model from huggingface
model = AutoModelForCausalLM.from_pretrained(
    'sarvamai/OpenHathi-7B-Hi-v0.1-Base',
    load_in_4bit=True,                                                            # loading the model with 4 bit precision, so as to show the example on Colab GPU
    device_map='auto'
)
tokenizer = AutoTokenizer.from_pretrained('sarvamai/OpenHathi-7B-Hi-v0.1-Base')
tokenizer.pad_token = tokenizer.eos_token

config.json:   0%|          | 0.00/667 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/936 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/968k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# setting the model generation configuration
generation_config = GenerationConfig(
    do_sample = True,
    temperature = 0.8,
    repetition_penalty = 1.5,
    max_new_tokens = 1
)

In [ ]:
inputs = tokenizer("I'll bet the video game is alot more fun than the film.\Sentiment:", return_tensors="pt")
print(inputs)
input_ids = inputs["input_ids"].to('cuda')

{'input_ids': tensor([[    1,   306, 29915,   645,  1010,   278,  4863,  3748,   338,   394,
           327,   901,  2090,  1135,   278,  2706,  7790, 29903,   296,  2073,
         29901]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
# passing the prompt tokens to the generative model to generate the response tokens
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        attention_mask=torch.ones_like(input_ids),
        generation_config=generation_config)

In [ ]:
# decode the model generated tokens, and printing the generated output.
output_text = tokenizer.decode(generation_output[0].cuda(), skip_special_tokens=True).strip()
print(output_text)

I'll bet the video game is alot more fun than the film.\Sentiment: negative


In [1]:
import pandas as pd

splits = {'train': 'pair-class/train-00000-of-00001.parquet', 'dev': 'pair-class/dev-00000-of-00001.parquet', 'test': 'pair-class/test-00000-of-00001.parquet'}
df = pd.read_parquet("hf://datasets/sentence-transformers/all-nli/" + splits["train"])

df.head()

,premise,hypothesis,label
0,A person on a horse jumps over a broken down a...,A person is training his horse for a competition.,1
1,A person on a horse jumps over a broken down a...,"A person is at a diner, ordering an omelette.",2
2,A person on a horse jumps over a broken down a...,"A person is outdoors, on a horse.",0
3,Children smiling and waving at camera,They are smiling at their parents,1
4,Children smiling and waving at camera,There are children present,0
